In [2]:
import numpy as np
from scipy.io.wavfile import read
import re
import os
import MFCC
from sklearn.model_selection import train_test_split
import pickle
import gmmhmm

ImportError: cannot import name 'logsumexp' from 'sklearn.utils.extmath' (C:\Anaconda3\lib\site-packages\sklearn\utils\extmath.py)

## Problem 1

Write a function which accepts a GMMHMM in the format above as well as an integer n_sim, and which simulates the GMMHMM process, generating n_sim different observations. Do so by implementing the following function declaration. 

In [57]:
def sample_gmmhmm(gmmhmm, n_sim, K=3):
    """
    Simulate sampling from a GMMHMM.

    Returns
    -------
    states : ndarray of shape (n_sim,)
        The sequence of states
    obs : ndarray of shape (n_sim, K)
        The generated observations (column vectors of length K)
    """
    states = []
    obs = []
    
    states.append(np.choice(np.arange(len(gmmhmm[1]))))
    
    for i in range(len(gmmhmm[0])):
        state_index = 
        sample_component = np.argmax(np.random.multinomial(1, weights[i,:]))
        obs.append(np.random.multivariate_normal(means[1, sample_component, :], covars[1, sample_component, :, :], size=n_sim))

    return states, obs

In [90]:
A = np.array([[.65, .35], [.15, .85]])
pi = np.array([.8, .2])
weights = np.array([[.7, .2, .1], [.1, .5, .4]])
means1 = np.array([[0., 17., -4.], [5., -12., -8.], [-16., 22., 2.]])
means2 = np.array([[-5., 3., 23.], [-12., -2., 14.], [15., -32., 0.]])
means = np.array([means1, means2])
covars1 = np.array([5*np.eye(3), 7*np.eye(3), np.eye(3)])
covars2 = np.array([10*np.eye(3), 3*np.eye(3), 4*np.eye(3)])
covars = np.array([covars1, covars2])
gmmhmm1 = [A, weights, means, covars, pi]

sample_gmmhmm(gmmhmm1, 4)

array([[[ -1.8450434 ,   5.7009968 ,  24.43413469],
        [ -2.76480275,  -0.82394218,  23.29464013]],

       [[ -9.17916328,   7.76791906,  24.37735337],
        [ -1.91638941,  -1.76444104,  19.39342479]],

       [[ 10.56532962, -30.47162504,  -1.12894249],
        [ 14.54477878, -29.27939992,  -1.32846158]],

       [[ 14.13692134, -34.06077811,   0.79578395],
        [ 14.33210332, -32.42904506,   1.2724842 ]]])

## Problem 2 

Obtain 30 (or more) recordings for each of the words/phrases mathematics, biology,
political science, psychology, and statistics. These audio samples should be 2 seconds in
duration, recorded at a rate of 44100 samples per second, with samples stored as 16-bit signed
integers in WAV format. Load the recordings into Python using scipy.io.wavfile.read.

In [72]:
bio_samples = []
math_samples = []
poly_sci_samples = []
stats_samples = []
for i in range(1, 31):
    bio_samples.append(read("Samples/Biology ({}).wav".format(i))[1])
    math_samples.append(read("Samples/Mathematics ({}).wav".format(i))[1])
    poly_sci_samples.append(read("Samples/PoliticalScience ({}).wav".format(i))[1])
    stats_samples.append(read("Samples/Statistics ({}).wav".format(i))[1])

In [87]:
print(math_samples[0].shape)

(88200,)


## Problem 3

Partition each list of MFCCs into a training set of 20 samples, and a test set of
the remaining 10 samples.

Using the training sets, train a GMMHMM on each of the words from the previous problem
with at least 10 random restarts, keeping the best model for each word (the one with the highest
log-likelihood). This process may take several minutes. Since you will not want to run this
more than once, you will want to save the best model for each word to disk using the pickle
module so that you can use it later.

In [81]:
bio_train, bio_test = train_test_split(bio_samples, train_size=20, test_size=10)
math_train, math_test = train_test_split(math_samples, train_size=20, test_size=10)
poly_sci_train, poly_sci_test = train_test_split(poly_sci_samples, train_size=20, test_size=10)
stats_train, stats_test = train_test_split(stats_samples, train_size=20, test_size=10)

In [91]:
def initialize(n):
    """
    takes in number of states and creates random 
    initial state distribution and transition matrix
    
    inputs:
        n: number of states
        
    outputs:
        startprob: initial state distribution
        transmat: transition matrix
    """
    # create initial state distribution
    startprob = np.random.rand(n)
    startprob /= np.sum(startprob)
    
    # create transition matrix
    transmat = np.random.rand(n,n)
    transmat = (transmat.T/np.sum(transmat, axis=1)).T
    
    # return results
    return startprob, transmat

In [93]:
startprob, transmat = initialize(5)
model = gmmhmm.GMMHMM(n_components=5, n_mix=3, transmat=transmat, startprob=startprob, cvtype='diag')
# these values for covars_prior and var should work well for this problem
model.covars_prior = 0.01
model.fit(bio_train, init_params='mc', var=0.1)
print(model.logprob)

AttributeError: 'list' object has no attribute 'GMMHMM'

## Problem 4

Classify the 10 test samples for each word. How does your system perform?
Which words are the hardest to correctly classify? Make a dictionary containing the accuracy
of the classification of your five testing sets. Specifically, the words/phrases will be the keys,
and the values will be the percent accuracy.

These are very positive results! Biology had the worst results and after looking closer it appears biology was misclassified as psychology three times, causing the lower accuracy. This is not too surprising since the two words have some similarities. Psychology was the only other word to not have perfect accuracy and was misclassified as biology once, again this is not surprising escpecially since biology was classified as psychology. The rest had perfect scores! Very cool. 